# Assignment 1: System Perspective

In [3]:
import Pkg
Pkg.activate(normpath(joinpath(@__DIR__, ".")))
Pkg.resolve()
Pkg.instantiate()

using Revise
using OPF.Utils
using OPF.A1

  Activating project at `~/dev/DTUElectricityMarkets2025/assignment1`
  No Changes to `~/dev/DTUElectricityMarkets2025/assignment1/Project.toml`
  No Changes to `~/dev/DTUElectricityMarkets2025/assignment1/Manifest.toml`


# Preparation

## Market Clearing as an optimization problem
The market operator clears the market by maximizing “social welfare”,
which is the area between the supply and demand curves.

The point where the demand and supply curves intersect is the **market clearing point**, or equilibrium point. At this point demands are happy because they get the energy they want at a price lower or equal to what they were willing to pay, and suppliers are happy because they get the price they want or more for the energy they supply.

The market clearing point is the point where both the **market clearing quantity** and **market clearing price** are determined. It can be found by solving an optimization problem, which is formulated as a simple linear program. 

NB: we consider here uniform pricing, i.e., all suppliers are **paid the same price** for the energy they supply, and all consumers pay the same price for the energy they consume.

$$
\begin{align}
\max_{\mathbf{p}^G, \mathbf{p}^D} \quad & \sum_{i=1}^{N_D} \pi_i^D p_i^D - \sum_{j=1}^{N_G} \pi_j^G p_j^G \\
\text{subject to} \quad & \sum_{j=1}^{N_G} p_j^G - \sum_{i=1}^{N_D} p_i^D = 0 \\
& 0 \leq p_i^D \leq P_i^D, \quad i = 1, \ldots, N_D \\
& 0 \leq p_j^G \leq P_j^G, \quad j = 1, \ldots, N_G
\end{align}
$$

Here:
- $N_D$ is the number of demand nodes
- $N_G$ is the number of generation nodes
- $p_i^D$ is the *elastic* scheduled power demand magnitude at node $i$
- $p_j^G$ is the scheduled power generation magnitude at node $j$
- $\pi_i^D$ is the demand bid price at node $i$
- $\pi_j^G$ is the generation bid price at node $j$
- $P_i^D$ is the maximum demand that can be offered at node $i$
- $P_j^G$ is the maximum generation that can be offered at node $j$

We can also write this as a minimization problem and in vector form:

$$
\begin{align}
\min_{\mathbf{p}^G, \mathbf{p}^D} \quad & \boldsymbol{\pi}^G \cdot \mathbf{p}^G - \boldsymbol{\pi}^D \cdot \mathbf{p}^D \\
\text{subject to} \quad & \mathbf{p}^G - \mathbf{p}^D = 0 \\
& 0 \leq \mathbf{p}^D \leq \mathbf{P}^D \\
& 0 \leq \mathbf{p}^G \leq \mathbf{P}^G
\end{align}
$$

Where $(\cdot)$ is the dot product and $\mathbf{p}^G$ and $\mathbf{p}^D$ are vectors of scheduled power generation and demand, respectively.

The constraint Eq. 2 ensures demand and generation are in balance.
The constraints Eq. 3 and Eq. 4 ensure that the scheduled power demand and generation are kept within their respective limits.

## Data collection and preparation

In order to solve a proper version of this optimization problem we need somewhat realistic pricing, generation and demand data. 

The assignment description is quite vague about data collection and preparation. It provides you some resources, but it does not explain how to use them. It is up to you to figure that out. I suppose from a pedagogical perspective this can either be a good or a bad thing. 

Collecting, curating and preparing data is an important, but time-consuming task in my experience. It doesn't hurt to become acquainted with it right now while you are still in the safety net of your studies. Real data is often (read: always) messy, incomplete, inconsistent and often not even correct! Dealing with this is a skill that you will need in your future career.

In my personal view it is not necessary to get to the core of the assignment, which is about formulating and solving a market clearing problem. The instructors *could* have saved you some time by providing you with a proper dataset, but didn't. So we have to spent some time doing it ourselves. I guess if you worked in a group you could draw straws ;)

I am lazy, so I used our AI overlord to transcribe PDF files to CSV. You can find these under `assignment1/data/`.

## Assumptions

Additionaly the following assumptions are made (from the assignement description):

| Assumption | Implication for optimization |
|------------|-------------|
| The production cost of renewable units is assumed to be zero.  | $\pi_j^G = 0$ for all renewable units $j$ |
| Reneweable units offer their forecasted capacity, meaning their offer quantities vary over time. | $P_j^G$ is time-dependent for all renewable units $j$ (so $P_{j,t}^G$)|


## Step 1: Copper plate, single hour

This is a very basic market clearing problem. We just need some load, generation and price data for a single hour. The data is provided in the `assignment1/data/` folder. This data is in CSV format, which we will read into a DataFrame.


In [4]:
demands = prepare_demands("data/demands.csv", hours=1:1)